                                  

```
# This is formatted as code
```

<h1>ICP -10<h1>
                                                  <h3>vvs murthy kolla<h3>
                                                  <h3>700729142<h3>

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import re 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder 
#Tokenization
from keras.preprocessing.text import Tokenizer 
 #Adding zeros or crop based on the length
from tensorflow.keras.preprocessing.sequence import pad_sequences
#Sequential Neural Network
from keras.models import Sequential 
 #For layers in Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical

In [ ]:
#mouting google drive
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
import pandas as pd

# Loading the dataset as a Pandas DataFrame
dataset = pd.read_csv('/content/gdrive/MyDrive/NN drive/Sentiment.csv')

# Selecting only the necessary columns 'text' and 'sentiment' from the dataset
mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]

# Keeping only the necessary columns in the dataset
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-7-d0e745dc69e5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-7-d0e745dc69e5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [ ]:
for idx, row in data.iterrows():
  #Removing all retweets
    row[0] = row[0].replace('rt', ' ') 
    max_fatures = 2000
     #spliting the sentence to max 2000 words
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values) 
 #obstracting values from the futire matrix
X = tokenizer.texts_to_sequences(data['text'].values)
#Padding the feature matrix
X = pad_sequences(X) 
#Dimension of the Embedded layer
embed_dim = 128 
 #Long short-term memory (LSTM) layer neurons
lstm_out = 196
def createmodel():
   #Sequential Neural Network
    model = Sequential()
     #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
     #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.add(Dense(3,activation='softmax')) 
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# printing(model.summary())
labelencoder = LabelEncoder() 
 #model fitting
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
#67% as training data, 33% as test data split
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) 
#mentioning batch size as 32
batch_size = 32 
#Functioning the  call to Sequential Neural Network
model = createmodel() 
 #verbose the higher, the more messages
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
 #evaluting the model as requried
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)

291/291 - 42s - loss: 0.8306 - accuracy: 0.6441 - 42s/epoch - 144ms/step
144/144 - 3s - loss: 0.7514 - accuracy: 0.6791 - 3s/epoch - 22ms/step
0.7513718008995056
0.6791175007820129


In [ ]:
#model metric evaluting
print(model.metrics_names) 

['loss', 'accuracy']


In [ ]:
#1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [ ]:
#save model for future use
model.save('sentimentAnalysis.h5') 

In [ ]:
from keras.models import load_model 
model= load_model('sentimentAnalysis.h5') 

In [ ]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [ ]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 270ms/epoch - 270ms/step
[0.72844136 0.10584743 0.16571125]
Neutral


 Apply GridSearchCV on the source code provided in the ***class***

In [ ]:

#importing Keras classifier
from keras.wrappers.scikit_learn import KerasClassifier 
#importing Grid search CV
from sklearn.model_selection import GridSearchCV 
#initiating model to test performance by applying multiple hyper parameters
model = KerasClassifier(build_fn=createmodel,verbose=2) 
#hyper parameter batch_size
batch_size= [10, 20, 40] 
#hyper parameter no. of epochs
epochs = [1, 2] 
#creating dictionary for batch size, no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} 
#Applying dictionary with hyper parameters
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
 #Fitting the model 
grid_result= grid.fit(X_train,Y_train)
# summarize results
#best score, best hyper parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 

<ipython-input-16-6c99b49150f4>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 89s - loss: 0.8275 - accuracy: 0.6466 - 89s/epoch - 120ms/step
186/186 - 3s - loss: 0.7607 - accuracy: 0.6676 - 3s/epoch - 18ms/step
744/744 - 82s - loss: 0.8253 - accuracy: 0.6473 - 82s/epoch - 111ms/step
186/186 - 3s - loss: 0.7795 - accuracy: 0.6676 - 3s/epoch - 15ms/step
744/744 - 86s - loss: 0.8231 - accuracy: 0.6434 - 86s/epoch - 116ms/step
186/186 - 2s - loss: 0.7761 - accuracy: 0.6686 - 2s/epoch - 13ms/step
744/744 - 84s - loss: 0.8271 - accuracy: 0.6425 - 84s/epoch - 113ms/step
186/186 - 2s - loss: 0.7908 - accuracy: 0.6738 - 2s/epoch - 12ms/step
744/744 - 84s - loss: 0.8205 - accuracy: 0.6451 - 84s/epoch - 113ms/step
186/186 - 2s - loss: 0.7877 - accuracy: 0.6615 - 2s/epoch - 12ms/step
Epoch 1/2
744/744 - 88s - loss: 0.8231 - accuracy: 0.6426 - 88s/epoch - 119ms/step
Epoch 2/2
744/744 - 83s - loss: 0.6856 - accuracy: 0.7103 - 83s/epoch - 112ms/step
186/186 - 2s - loss: 0.7281 - accuracy: 0.6859 - 2s/epoch - 13ms/step
Epoch 1/2
744/744 - 85s - loss: 0.8195 - accuracy